In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [3]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [3]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') < lit('2018-11-12'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

### Built-in functions


In [4]:
from pyspark.sql import functions
print(dir(functions))

['Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_udf', '_get_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_column', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aggregate', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'array_distinct', 'array_except', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_remove', 'array_repeat', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'asc', 'asc_nulls_first', 'asc_nulls_last', 'ascii', 'asin', 'asinh', 'assert_true', 'atan', 'atan2', 'atanh', 'a

In [5]:
from pyspark.sql.functions import *
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str, pos, len)
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. versionadded:: 1.5.0
    
    Notes
    -----
    The position is not zero based, but 1 based index.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]



In [6]:
rc.select(lower(col("Case Number"))).show()

+------------------+
|lower(Case Number)|
+------------------+
|          hy411648|
|          hy411615|
|          jc213529|
|          hy411595|
|          hy411610|
|          hy411435|
|          hy411629|
|          hy411605|
|          hy411654|
|          jc212333|
|          hy411662|
|          hy411626|
|          hy411632|
|          hy411566|
|          hy411601|
|          hy411489|
|          hy411656|
|          hy410094|
|          hy411388|
|          hy411568|
+------------------+
only showing top 20 rows



In [7]:
rc.select(lower(col("Case Number")),upper(col("Case Number")),substring(col("Case Number"),1,4)).show(5)

+------------------+------------------+----------------------------+
|lower(Case Number)|upper(Case Number)|substring(Case Number, 1, 4)|
+------------------+------------------+----------------------------+
|          hy411648|          HY411648|                        HY41|
|          hy411615|          HY411615|                        HY41|
|          jc213529|          JC213529|                        JC21|
|          hy411595|          HY411595|                        HY41|
|          hy411610|          HY411610|                        HY41|
+------------------+------------------+----------------------------+
only showing top 5 rows



**Display the Primary Type column in lower and upper characters, and the first 4 characters of the column**

In [8]:
rc.select(lower(col("Primary Type")),upper(col("Primary Type")),substring(col("Primary Type"),1,4)).show(5)

+-------------------+-------------------+-----------------------------+
|lower(Primary Type)|upper(Primary Type)|substring(Primary Type, 1, 4)|
+-------------------+-------------------+-----------------------------+
|            battery|            BATTERY|                         BATT|
|              theft|              THEFT|                         THEF|
|              theft|              THEFT|                         THEF|
|          narcotics|          NARCOTICS|                         NARC|
|            assault|            ASSAULT|                         ASSA|
+-------------------+-------------------+-----------------------------+
only showing top 5 rows



### Numeric functions
##### Show the oldest date and the most recent date

In [16]:
rc.select(min(col("Date")),max(col("Date"))).show()

+-------------------+-------------------+
|          min(Date)|          max(Date)|
+-------------------+-------------------+
|2001-01-01 00:00:00|2018-11-11 14:00:00|
+-------------------+-------------------+



### Date

#####  What is 3 days earlier that the oldest date and 3 days later than the most recent date?**

In [17]:
from pyspark.sql.functions import date_add,date_sub

In [18]:
help(date_add)

Help on function date_add in module pyspark.sql.functions:

date_add(start, days)
    Returns the date that is `days` days after `start`
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_add(df.dt, 1).alias('next_date')).collect()
    [Row(next_date=datetime.date(2015, 4, 9))]



In [20]:
rc.select(date_add(max(col("Date")),3)).show()

+----------------------+
|date_add(max(Date), 3)|
+----------------------+
|            2018-11-14|
+----------------------+



In [22]:
rc.select(date_sub(min(col("Date")),3)).show()

+----------------------+
|date_sub(min(Date), 3)|
+----------------------+
|            2000-12-29|
+----------------------+



##  NOT USING CRIME CSV FILE

## Working with dates

In [23]:
#####33  we use to_date and to_timesatmp

In [27]:
rc.select(to_date(lit("2019-12-01"))).show(1)

+-------------------+
|to_date(2019-12-01)|
+-------------------+
|         2019-12-01|
+-------------------+
only showing top 1 row



In [29]:
##############   create DATAFRAME, create column , creating categories AND FILTERING IT   WITH WHEN...................

from pyspark.sql.functions import when
df = spark.createDataFrame([["2019-12-25"],['2020-12-25']],["Christmas"])
df.show()

+----------+
| Christmas|
+----------+
|2019-12-25|
|2020-12-25|
+----------+



In [31]:
df.dtypes

[('Christmas', 'string')]

In [33]:
########################################################3               to_date

df.select(to_date(col("Christmas"),"yyyy-MM-dd")).show()

+------------------------------+
|to_date(Christmas, yyyy-MM-dd)|
+------------------------------+
|                    2019-12-25|
+------------------------------+
only showing top 1 row



In [35]:
########################################################3               to_timestamp


df.select(to_timestamp(col("Christmas"),"yyyy-MM-dd")).show()

+-----------------------------------+
|to_timestamp(Christmas, yyyy-MM-dd)|
+-----------------------------------+
|                2019-12-25 00:00:00|
|                2020-12-25 00:00:00|
+-----------------------------------+



In [41]:
########################################################3               to_timestamp -------  another method


df.withColumn('Christmas',to_timestamp(col('Christmas'),'yyyy-MM-dd')).show()
df.dtypes

+-------------------+
|          Christmas|
+-------------------+
|2019-12-25 00:00:00|
|2020-12-25 00:00:00|
+-------------------+



[('Christmas', 'string')]

In [36]:
############   another format

df.select(to_timestamp(col("Christmas"),"yyyy/MM/dd")).show()

+-----------------------------------+
|to_timestamp(Christmas, yyyy/MM/dd)|
+-----------------------------------+
|                               null|
|                               null|
+-----------------------------------+



https://medium.com/expedia-group-tech/deep-dive-into-apache-spark-datetime-functions-b66de737950a

parse dates 

https://sparkbyexamples.com/spark/spark-date-functions-format-dates/

## joins

In [43]:
########  we will download police station dataset

In [44]:
!wget -O police_station.csv  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD



--2021-04-03 22:16:32--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: 'police_station.csv'

     0K .....                                                  3.89M=0.001s

2021-04-03 22:16:33 (3.89 MB/s) - 'police_station.csv' saved [5699]



In [13]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is 4234-2518

 Directory of C:\Users\ANTHONY\Desktop\important ipynb codes\SPARK

11-05-2021  12:17 PM    <DIR>          .
11-05-2021  12:17 PM    <DIR>          ..
11-05-2021  11:45 AM    <DIR>          .ipynb_checkpoints
11-05-2021  08:23 AM             4,980 0.RUN THIS FIRST.ipynb
11-05-2021  08:36 AM            37,871 1.Context , session and data import.ipynb
11-05-2021  12:17 PM           120,050 2. Schemas_Columns_rows_groupby_sort_unique.ipynb
11-05-2021  12:17 PM            84,258 3.Functions&date&joins.ipynb
03-04-2021  09:23 AM             4,046 Download_data.ipynb
03-04-2021  07:33 PM             2,294 FIRST_SAMPLE.ipynb
05-05-2021  09:05 PM    <DIR>          New Folder
19-08-2019  06:56 AM             5,699 police_station.csv
05-04-2021  10:03 AM            22,987 RDD.ipynb
02-04-2021  04:48 PM       171,057,223 reported-crimes.csv
02-04-2021  04:48 PM       171,057,223 rows.csv@accessType=DOWNLOAD
02-06-2018  02:19 AM    

In [15]:
from pyspark.sql.functions import to_timestamp,col,lit
ps = spark.read.csv('police_station.csv',header=True)
ps.show(5)

+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|          18|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.029| 1908086.527|41.90324165|-87.64335214|(41.9032416531, -

In [16]:
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [47]:
###  the CRIME DATASET HAS ONLY DISTRICT NUMBER,, NEED TO FIND DISTRICT NAME FOR THOSE... CAN BE DONE BY JOIN  ######

In [17]:
rc.select(col("District")).distinct().show()

+--------+
|District|
+--------+
|     009|
|     012|
|     024|
|    null|
|     031|
|     015|
|     006|
|     019|
|     020|
|     011|
|     025|
|     005|
|     003|
|     016|
|     018|
|     008|
|     022|
|     001|
|     014|
|     010|
+--------+
only showing top 20 rows



In [18]:
ps.select(col("DISTRICT")).distinct().show()

+------------+
|    DISTRICT|
+------------+
|           7|
|          15|
|          11|
|           3|
|           8|
|          22|
|          16|
|           5|
|          18|
|          17|
|           6|
|          19|
|          25|
|Headquarters|
|          24|
|           9|
|           1|
|          20|
|          10|
|           4|
+------------+
only showing top 20 rows



In [19]:
#############   rc district code have leading zeros, but other donot...
######   so we need to remove this

help(lpad)

Help on function lpad in module pyspark.sql.functions:

lpad(col, len, pad)
    Left-pad the string column to width `len` with `pad`.
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(lpad(df.s, 6, '#').alias('s')).collect()
    [Row(s='##abcd')]



In [20]:
ps.select(lpad(ps.DISTRICT, 3, "0")).show()

+--------------------+
|lpad(DISTRICT, 3, 0)|
+--------------------+
|                 Hea|
|                 018|
|                 019|
|                 020|
|                 022|
|                 024|
|                 025|
|                 001|
|                 002|
|                 003|
|                 004|
|                 005|
|                 006|
|                 007|
|                 008|
|                 009|
|                 010|
|                 011|
|                 012|
|                 ",C|
+--------------------+
only showing top 20 rows



In [21]:
########  create a new column with above formatting

ps=ps.withColumn("DISTRICT",lpad(ps.DISTRICT, 3, "0"))

ps.show(5)

+--------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+--------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|     Hea| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|     018|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.029| 1908086.527|41.90324165|-87.64335214|(41.9032416531, -...|
|     019|    T

In [66]:
rc.join(ps,ps.DISTRICT ==  rc.District,"left_outer").show(truncate=False)

+--------+-----------+-------------------+-----------------------+----+------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+------------+-------------+-----------------------------+--------+--------------+----------------------+-------+-----+-----+-------------------------------------------------------------------------------+------------+------------+------------+------------+------------+-----------+------------+-------------------------------+
|ID      |Case Number|Date               |Block                  |IUCR|Primary Type      |Description                                   |Location Description          |Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On            |Latitude    |Longitude    |Location                     |DISTRICT|DISTRICT NAME |ADDRESS               |CITY   

In [22]:
######33  too many columns------------- messy

rc.join(ps,ps.DISTRICT ==  rc.District,"left_outer").drop('ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION').show(truncate=False)

+--------+-----------+-------------------+-----------------------+----+------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+----+----------------------+--------+--------------+
|ID      |Case Number|Date               |Block                  |IUCR|Primary Type      |Description                                   |Location Description          |Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|Updated On            |DISTRICT|DISTRICT NAME |
+--------+-----------+-------------------+-----------------------+----+------------------+----------------------------------------------+------------------------------+------+--------+----+--------+----+--------------+--------+----+----------------------+--------+--------------+
|10224738|HY411648   |2015-09-05 13:30:00|043XX S WOOD ST        |0486|BATTERY           |DOMESTIC BATTERY SIMPLE                       |RESIDENCE              

In [23]:
rc.dtypes

[('ID', 'string'),
 ('Case Number', 'string'),
 ('Date', 'timestamp'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'string'),
 ('Domestic', 'string'),
 ('Beat', 'string'),
 ('District', 'string'),
 ('Ward', 'string'),
 ('Community Area', 'string'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'string'),
 ('Y Coordinate', 'string'),
 ('Year', 'string'),
 ('Updated On', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string'),
 ('Location', 'string')]

In [24]:
ps.columns

['DISTRICT',
 'DISTRICT NAME',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION']

In [ ]:
#######################                                                                     ################################3333
                            #  WHICH DAY HAS THE MOST NUMBER OF CRIMES  #######################
########################                                                                     ################################

In [25]:
########  get day, week, month from spark dataframe


x=rc.withColumn("Day", date_format(col("Date"), "E"))

In [26]:
x.show()

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|Day|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RE

In [27]:
x.groupBy("Day").count().orderBy("count").show(10)

+---+------+
|Day| count|
+---+------+
|Sun| 98284|
|Wed| 99459|
|Tue| 99958|
|Thu|100770|
|Mon|101311|
|Sat|102306|
|Fri|105872|
+---+------+



In [ ]:
############################################           OR             ################################################

In [117]:
help(dayofweek)

Help on function dayofweek in module pyspark.sql.functions:

dayofweek(col)
    Extract the day of the week of a given date as integer.
    
    .. versionadded:: 2.3.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(dayofweek('dt').alias('day')).collect()
    [Row(day=4)]



In [28]:

rc.select("Date", dayofweek(col("Date"))).show()

+-------------------+---------------+
|               Date|dayofweek(Date)|
+-------------------+---------------+
|2015-09-05 13:30:00|              7|
|2015-09-04 11:30:00|              6|
|2018-09-01 00:01:00|              7|
|2015-09-05 12:45:00|              7|
|2015-09-05 13:00:00|              7|
|2015-09-05 10:55:00|              7|
|2015-09-04 18:00:00|              6|
|2015-09-05 13:00:00|              7|
|2015-09-05 11:30:00|              7|
|2016-05-01 00:25:00|              1|
|2015-09-05 14:00:00|              7|
|2015-09-05 11:00:00|              7|
|2015-09-05 03:00:00|              7|
|2015-09-05 12:50:00|              7|
|2015-09-03 13:00:00|              5|
|2015-09-05 11:45:00|              7|
|2015-09-05 13:30:00|              7|
|2015-07-08 00:00:00|              4|
|2015-09-05 09:55:00|              7|
|2015-09-05 12:35:00|              7|
+-------------------+---------------+
only showing top 20 rows



In [30]:
rc.withColumn("Day", date_format(col("Date"), "E")).show()

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|Day|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RE

In [93]:
##################################  MOST FREQUENTLY reported NON-CRIMINAL ACTIVITY  ######################################

#####################################                                                 ####################################

In [99]:
rc.select("Primary Type").distinct().count()

35

In [106]:
rc.groupBy("Primary Type").count().orderBy("Primary Type").show(35)

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|                null|     1|
|               ARSON|  1221|
|             ASSAULT| 49239|
|             BATTERY|131247|
|            BURGLARY| 36290|
|CONCEALED CARRY L...|   143|
| CRIM SEXUAL ASSAULT|  3953|
|     CRIMINAL DAMAGE| 79989|
|CRIMINAL SEXUAL A...|    42|
|   CRIMINAL TRESPASS| 17723|
|  DECEPTIVE PRACTICE| 48812|
|            GAMBLING|   541|
|            HOMICIDE|   126|
|   HUMAN TRAFFICKING|    31|
|INTERFERENCE WITH...|  2940|
|        INTIMIDATION|   362|
|          KIDNAPPING|   528|
|LIQUOR LAW VIOLATION|   615|
| MOTOR VEHICLE THEFT| 29397|
|           NARCOTICS| 38057|
|      NON - CRIMINAL|    13|
|        NON-CRIMINAL|   104|
|NON-CRIMINAL (SUB...|     4|
|           OBSCENITY|   173|
|OFFENSE INVOLVING...|  6037|
|OTHER NARCOTIC VI...|    15|
|       OTHER OFFENSE| 46626|
|        PROSTITUTION|  2209|
|    PUBLIC INDECENCY|    27|
|PUBLIC PEACE VIOL...|  4503|
|         

In [ ]:
############  NON - CRIMINAL|    13|
#        NON-CRIMINAL|   104|
#           |NON-CRIMINAL (SUB...

In [31]:
y=rc.filter((col("Primary Type") == "NON-CRIMINAL")|(col("Primary Type") == "NON - CRIMINAL")|(col("Primary Type") == "NON-CRIMINAL (SUB SPECIFIED)"))
y.show()

+--------+-----------+-------------------+--------------------+----+--------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|  Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+--------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10233370|   HY421530|2015-09-13 10:45:00|  010XX S MENARD AVE|5114|NON - CRIMINAL|   FOID - REVOCATION|           RESIDENCE|  true|   false|1513

In [32]:
y.groupBy("Description").count().orderBy("count",ascending=False).show(10)

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|       LOST PASSPORT|   69|
|   FOID - REVOCATION|   38|
|CONCEALED CARRY L...|    4|
|NOTIFICATION OF S...|    3|
|      FOUND PASSPORT|    2|
|GUN OFFENDER NOTI...|    1|
+--------------------+-----+



In [33]:
###LOST PASSPORT|   69  is the answer